# RandomForest with scikit-learn

We want to explore how to use scikit-learn for RandomForest. This includes preparing the data, training the model, and evaluating and visualizing the results.

We will use the following modules for this:
* [sklearn.datasets](https://scikit-learn.org/stable/api/sklearn.datasets.html): Tools for using common datasets for ML or for generating synthetic data
* [sklearn.model_selection](https://scikit-learn.org/stable/api/sklearn.model_selection.html): Tools for data splitting, cross-validation, and parameter tuning
* [sklearn.tree](https://scikit-learn.org/stable/api/sklearn.tree.html) and [sklearn.ensemble](https://scikit-learn.org/stable/api/sklearn.ensemble.html): Collection of tree- and ensemble-based models for regression and classification
* [sklearn.metrics](https://scikit-learn.org/stable/api/sklearn.metrics.html): Collection of various metrics for model evaluation

**In general, also refer to the comprehensive [User Guide on decision trees](https://scikit-learn.org/stable/modules/tree.html) and [User Guide on ensemble models](https://scikit-learn.org/stable/modules/ensemble.html) from scikit-learn.**

In [ ]:
# Just in case we need help
# Import bia-bob as a helpful Python & Medical AI expert
from bia_bob import bob
import os

bob.initialize(
    endpoint='https://kiara.sc.uni-leipzig.de/api/v1', 
    model="vllm-llama-4-scout-17b-16e-instruct",
    system_prompt=os.getenv('SYSTEM_PROMPT_MEDICAL_AI')
)

In [ ]:
%bob Who are you ? Just 1 sentence!

## RandomForest for Regression

### Data preparation

Now we want to use a RandomForest to solve the regression problem for the [scikit-learn diabetes dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset).

Let's load the data again:

In [ ]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()

In [ ]:
# Let's get our X,y first
X = diabetes.data
y = diabetes.target
# And here we also have proper labels for our features
feature_labels = diabetes.feature_names

In [ ]:
print("Type X:", type(X))
print("Shape X:", X.shape)
print("First X:", X[0])
print("Labels X:", feature_labels)

In [ ]:
print("Type y:", type(y))
print("Shape y:", y.shape)
print("First y:", y[0])

### Train-test split

Now we split the data for training and model testing / evaluation in order to verify the generalization of the trained model on unknown data. For this purpose, we can use [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [ ]:
from sklearn.model_selection import train_test_split

# Split: 80% training, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("training data:", X_train.shape, "training labels:", y_train.shape)
print("test data:", X_test.shape, "test labels:", y_test.shape)

### Select and train a model

* We import the model [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) and initialize the corresponding Python object using `()` **while also setting specific model parameters**
* Model training is started using the method [`.fit()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.fit) 
* We pass the training data to this method, divided into features `X_train` and target `y_train`
* The model parameters are now adjusted to predict the target as accurately as possible based on the associated feature

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize model with model parameters
# - n_estimators: number of decision trees in the forest
# - max_depth: maximum depth of a tree
model = RandomForestRegressor(
    n_estimators=5,
    max_depth=None
)

In [ ]:
# Supervised training - “Fitting” the model to the training data with known labels
model.fit(X_train, y_train)

### Prediction and evaluation

We now want to check how well the model can predict the target for previously unknown features.

* The prediction is performed using the method [`.predict()`](https://scikit-learn.org/1.8/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.predict) – almost all models and algorithms in scikit-learn have this method.
* We pass the features of the test data to this method and receive the corresponding predictions of the target.

In [ ]:
# Prediction on unseen test data
y_pred = model.predict(X_test)

We can now compare the predictions with the known target values of the test data and derive metrics for evaluating the model quality.

* A metric provided by RandomForestRegressor itself is the so-called `r2` score (coefficient of determination of a regression), which evaluates the overall goodness of fit
* Other suitable regression metrics are described, for example, in the [User Guide - Regression Metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics)
* We also choose the [Mean Absolute Error (MAE)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html#sklearn.metrics.mean_absolute_error)
* In general, it makes sense to calculate several metrics in order to get a better impression of the model quality

In [ ]:
from sklearn.metrics import mean_absolute_error

# Metrics for determining model quality

# r^2 score, between 0.0 and 1.0, higher is better
r2_score = model.score(X_test, y_test)
print(f"r^2 score on test data: {r2_score:.3f}")

# Calculate mean absolute error (MAE), the best is 0.0
mse = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE) on test data: {mse:.3f}")

### Parameter tuning

At the moment, we have simply estimated the values for the model parameters `n_estimators` and `max_depth` and obtained an evaluation result for this combination. But how can we be sure that this is actually the best choice, or whether we can still improve the model quality?

In [ ]:
# We can check all available parameters again
model.get_params()

Since trial and error would be very time-consuming with a large number of available parameters and combinations, we should use an automated approach that employs [scikit-learn hyper-parameter tuning methods](https://scikit-learn.org/stable/modules/grid_search.html).

We want to apply a basic grid search over specified parameter values using [`sklearn.model_selection.GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid we want to search
param_grid = {
    'n_estimators': [1, 25, 50, 200],
    'max_depth': [None, 5, 25, 50],
}

# Initialize GridSearchCV with the following parameters
# - estimator: the model we want to tune
# - param_grid: the parameter grid to search from
# - scoring: score used to determine the model quality for given parameters
# - cv: cross-validation data splitting strategy (n-fold)
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    scoring='r2',
    cv=5
)

In [ ]:
# Perform grid search, may take a while
grid_search.fit(X_train, y_train)

In [ ]:
# Print best parameter combination with the best score
print("Best parameters for training data:", grid_search.best_params_)
print("Best score for training data:", grid_search.best_score_)

# Get the best model with the best parameter choice
best_model = grid_search.best_estimator_

As we can see, setting `max_depth=5` and `n_estimators=200` delivered the best results for the training data.

Let's evaluate the predictions of this best model with our test data.

In [ ]:
# Prediction on unseen test data
y_pred_best = best_model.predict(X_test)

# r^2 score, between 0.0 and 1.0, higher is better
r2_score = best_model.score(X_test, y_test)
print(f"r^2 score on test data: {r2_score:.3f}")

# Calculate mean absolute error (MAE), the best is 0.0
mse = mean_absolute_error(y_test, y_pred_best)
print(f"Mean Absolute Error (MAE) on test data: {mse:.3f}")

### Explainability

### Tree insights

Now that we have trees built inside our RandomForest, we can examine separate trees by plotting them.

In [ ]:
from sklearn import tree

# Plot the first tree from our RandomForest with limited depth
tree.plot_tree(best_model.estimators_[0],
               max_depth=1,
               feature_names = feature_labels)

#### Feature importance

Now that we have an example with more than one feature, we can also evaluate the significance / importance of the features for the regression task. This helps to understand which features contributed to the prediction and to what extent. This information can then be used for a more refind feature selection. 

For this, some models in scikit-learn provide the attribute `feature_importances_` (impurity-based, good for low cardinality features). In addition, the more sophisticated method [`sklearn.inspection.permutation_importance`](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html) is available.

In [ ]:
import numpy as np
import pandas as pd

# Extract feature importances from the best model, store sorted in a pandas Series
importances = pd.Series(best_model.feature_importances_, index=feature_labels).sort_values(ascending=False)

# Get the standard deviations of importances across trees as a measure of the variability
importances_std = np.std([tree.feature_importances_ for tree in best_model.estimators_], axis=0)

Now we can plot the feature importances, including the standard deviations as error bars

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
importances.plot(kind='bar', yerr=importances_std)
plt.xlabel('Feature')
plt.ylabel('Importance, with error bars (std)')
plt.title('Feature Importance for RandomForestRegressor')
plt.show()

## Exercise: Classification with RandomForest

Now it's your turn to apply a RandomForest to a classification problem, using the [scikit-learn breast cancer diagnostic dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset).

This data contains features computed from 569 digitized images of a breast mass, which shall be used to predict two classes: WDBC-Malignant, or WDBC-Benign. 

For example, you can use the following objects and methods:
* [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) as model
* [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) or [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) for parameter optimization
* [Classification Metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics) to evaluate the model quality
* [`permutation_importance`](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html) for explainability

**If you get stuck, remember that our assistant bia-bob is available and very happy to help you.**

### Data preparation

In [ ]:
from sklearn.datasets import load_breast_cancer

breast_cancer = load_breast_cancer()
dir(breast_cancer)

In [ ]:
# ToDo: understand the data better

### Train-test split

In [ ]:
# ToDo: create data for training and test

### Select and train a model

In [ ]:
# ToDo: initialize and train a classification model with parameter optimization

### Prediction and evaluation

In [ ]:
# ToDo: evaluate the model quality